<a href="https://colab.research.google.com/github/LIYunzhe1408/GPT_Transformer_spell_out/blob/main/build_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download the dataset
Download a tiny dataset and peak it

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-11-17 05:41:07--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.08s   

2025-11-17 05:41:08 (13.8 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



## Vocab & Tokenization

Create the vocabulary of this dataset.

All the unique characters in this text. Possible characters the model can see and imitate.

In [ ]:
characters = sorted(list(set(text)))
vocab_size = len(characters)
print(''.join(characters))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Convert the raw text to some sequence of integers according to the vocabulary of all possible elements.

> Short sequences of integers with very large vocabularies V.S. Long sequences of integers with small vocabularies

`hii there`
* Basic mapping with all characters: 2, 46, 46, 1, 52, 2, 12, 54, 12 with only 65 vocabularies.
* Tiktoken: 71, 4178, 612 with 50257 vocabularies.



In [ ]:
itos = {i: ch for i, ch in enumerate(characters)}
stoi = {ch: i for i, ch in itos.items()}
encode = lambda string: [stoi[ch] for ch in string] # Encoder: take a string, output a list of integers
decode = lambda integers: ''.join([itos[i] for i in integers]) # Decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii, there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii, there


In [ ]:
# Encode the text and store it in to a torch.tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


## Split train and val
- First 90% of the `training` dataset
- Last 10% as the `validation` dataset

In [ ]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data   = data[n:]
print(len(train_data), len(val_data))

1003854 111540


## Data Loader
1. Time dimension
  - How many characters the model can receive: block size.
  - The model can generate answers starting from as little as one context character to block_size length. Because during the training process, the model is set to train across limit. Over that limit, the model will chunk the data and will not be able to receive.

In [ ]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(context, "--->", target.item())

tensor([18]) ---> 47
tensor([18, 47]) ---> 56
tensor([18, 47, 56]) ---> 57
tensor([18, 47, 56, 57]) ---> 58
tensor([18, 47, 56, 57, 58]) ---> 1
tensor([18, 47, 56, 57, 58,  1]) ---> 15
tensor([18, 47, 56, 57, 58,  1, 15]) ---> 47
tensor([18, 47, 56, 57, 58,  1, 15, 47]) ---> 58


2. Batch dimension
  - Multiple chunks of text will stack up in a single tensor. This is done for efficiency to keep the gpus busy to process data parallel.
  - Batch size: how many independent sequences will we process in parallel?
  - Block size: what is the maximum context length for predictions?

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  index_x = torch.randint(len(data) - block_size, (batch_size,))
  xb = torch.stack([data[i:i+block_size] for i in index_x])
  yb = torch.stack([data[i+1:i+block_size+1] for i in index_x])
  return xb, yb

xb, yb = get_batch("train")
print("Inputs: ")
print(xb.shape)
print(xb)
print("Targets: ")
print(yb.shape)
print(yb)

# Given multiple/single character as context
# Output one next character
for b in range(batch_size):
  print(f"The {b}-th sample in this batch")
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"When the input is {context} ---> targert is {target.item()}")
  print("--------------------------------------")

Inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
The 0-th sample in this batch
When the input is tensor([24]) ---> targert is 43
When the input is tensor([24, 43]) ---> targert is 58
When the input is tensor([24, 43, 58]) ---> targert is 5
When the input is tensor([24, 43, 58,  5]) ---> targert is 57
When the input is tensor([24, 43, 58,  5, 57]) ---> targert is 1
When the input is tensor([24, 43, 58,  5, 57,  1]) ---> targert is 46
When the input is tensor([24, 43, 58,  5, 57,  1, 46]) ---> targert is 43
When the input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) ---> targert is 39
--------------------------------------
The 1-th sample in this ba

# Baseline
- bigram language model
- loss
- generation

In [ ]:
print(xb)
print(xb.shape)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # Each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    # idx and targets are both (B, T) tensors of integers
    logits = self.token_embedding_table(idx) # (B, T, C)

    # Just inference
    if targets is None:
      return logits, None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
      return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx: (B, T)
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      # For Biagram model, we just see the last time step
      logits = logits[:, -1, :] # becomes (B, C)
      # Apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # Sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # Append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
idx = m.generate(idx, max_new_tokens=100)[0]
print(decode(idx.tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [ ]:
# Train the model
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
for step in range(10000):
  # Sample a batch of data
  xb, yb = get_batch("train")

  # Evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.4491610527038574


In [ ]:
idx = torch.zeros((1, 1), dtype=torch.long)
idx = m.generate(idx, max_new_tokens=300)[0]
print(decode(idx.tolist()))


Se I m tis be h bed BRe hucul bew Jal s helas!

I's, golld.
Thondreed ait?
Pr t:
Tod.

IIViray:
ADuto beebof brs ICE sar t-by h k f mbus.
Yo wha moan
HENThitigo thes;
Awnay wenthunt! oofay nt thet o--
Mestheaitloraimf name nseat hilom moriss thethe t o selobe or sh oer conture wf imbunosoudedsuiss e


## Self attention
Currently, each token is not talking to each other, but we want them to.
We would like to couple them

However, we don't want the fifth position talking to the sixth, seventh, ... positions that behind it.
It can only talk to the fourth, third, second, and the first.
So the information only flows from the previous context to the current time step. We can not get the information from the future because we are trying to predict the future.

One simplest is to sum or average the preceeding positions, but sum or average are weak interaction. We lose a ton of information of spatial arrangement of tokens. But that's okay for now. We will see how to bring this information back later.

In [2]:
import torch

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

Version 1

In [6]:
# Instead of fumbling around, we copy and paste the code snippet here
# we want the x[b, t] = mean_{i<=t} x[b, i]
x_bag_of_words = torch.zeros((B, T, C))
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1]
    x_bag_of_words[b, t] = torch.mean(xprev, dim=0)

torch.Size([4, 8, 2])

In [7]:
x[0]

tensor([[-0.7962,  1.9303],
        [-0.9944,  0.1796],
        [-0.7439, -0.7859],
        [ 0.0915,  1.6276],
        [-1.8162, -1.0358],
        [ 1.1907, -0.3382],
        [ 0.6329,  1.9129],
        [ 0.2930,  2.1488]])

In [8]:
x_bag_of_words[0]

tensor([[-0.7962,  1.9303],
        [-0.8953,  1.0550],
        [-0.8448,  0.4414],
        [-0.6107,  0.7379],
        [-0.8518,  0.3832],
        [-0.5114,  0.2630],
        [-0.3479,  0.4987],
        [-0.2678,  0.7049]])

Version 2

In [17]:
# Use matrix multiplication to be more efficient
a = torch.tril(torch.ones((3, 3)))
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print("a-----")
print(a)
print("b-----")
print(b)
print("c-----")
print(c)

a-----
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b-----
tensor([[3., 3.],
        [6., 7.],
        [6., 7.]])
c-----
tensor([[3.0000, 3.0000],
        [4.5000, 5.0000],
        [5.0000, 5.6667]])
